In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import os
from pathlib import Path
import IPython.display as ipd

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "."]).decode("utf8"))

from scipy.io import wavfile # reading the wavfile

# Any results you write to the current directory are saved as output.

Dockerfile
LICENSE
README.md
log_spect.ipynb
prepare_data_set.ipynb
sample_submission.7z
sample_submission.7z.1
sample_submission.csv
silence_is_golden.csv
test
test.7z
train
train_files.csv



In [13]:
print(check_output(["ls", "./train"]).decode("utf8"))

folders = os.listdir("./train/audio")
print(folders)

LICENSE
README.md
audio
testing_list.txt
validation_list.txt

['bird', 'on', 'go', 'zero', 'two', 'stop', 'cat', 'left', 'happy', 'four', 'no', 'up', 'one', 'dog', 'off', 'right', 'tree', 'wow', '_background_noise_', 'six', 'sheila', 'eight', 'marvin', 'down', 'three', 'five', 'house', 'bed', 'nine', 'yes', 'seven']


## Some starter script to help with the data processing

In [16]:
train_audio_path = './train/audio'

train_labels = os.listdir(train_audio_path)
train_labels.remove('_background_noise_')
print('Number of labels: ', len(train_labels))

labels_to_keep = ['yes', 'no', 'up', 'down', 'left',
                  'right', 'on', 'off', 'stop', 'go', 'silence']

train_file_labels = dict()
for label in train_labels:
    files = os.listdir(train_audio_path + '/' + label)
    for f in files:
        train_file_labels[label + '/' + f] = label

train = pd.DataFrame.from_dict(train_file_labels, orient='index')
train = train.reset_index(drop=False)
train = train.rename(columns={'index': 'speaker_file', 0: 'folder'})
train = train[['folder', 'speaker_file']]
train = train.sort_values('speaker_file')
train = train.reset_index(drop=True)

def remove_label_from_file(label, fname):
    return fname[len(label)+1:]

train['speaker_file'] = train.apply(lambda x: remove_label_from_file(*x), axis=1)
train['file_path'] = train['folder'] + '/' + train['speaker_file']
train['label'] = train['folder'].apply(lambda x: x if x in labels_to_keep else 'unknown')
train['all_labels'] = train['folder']
print(train.shape)

train.to_csv('train_files.csv')

Number of labels:  30
(64721, 5)


In [17]:
train.sample(5)

folder           speaker_file                    file_path    label  \
35644     on  efa16ffd_nohash_0.wav     on/efa16ffd_nohash_0.wav       on   
32158    off  7ab5b8f7_nohash_0.wav    off/7ab5b8f7_nohash_0.wav      off   
11456  eight  e99d36c8_nohash_1.wav  eight/e99d36c8_nohash_1.wav  unknown   
42405  seven  cae62f38_nohash_0.wav  seven/cae62f38_nohash_0.wav  unknown   
15379   four  9735764a_nohash_0.wav   four/9735764a_nohash_0.wav  unknown   

      all_labels  
35644         on  
32158        off  
11456      eight  
42405      seven  
15379       four

## You can listen to the audio directly from the Kernel

In [32]:
ipd.Audio(str(train_audio_path) + '/_background_noise_/doing_the_dishes.wav')

In [41]:
sample_rate, audio = wavfile.read(str(train_audio_path) + '/_background_noise_/doing_the_dishes.wav')
audio.shape

/usr/local/lib/python3.5/dist-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


(1522930,)

In [45]:
print('Before suppression',audio.mean())
audio_suppressed = audio * 0.1
print('After suppression', audio_suppressed.mean())

wavfile.write(str(train_audio_path) + '/_background_noise_/doing_the_dishes_suppressed.wav', sample_rate, audio_suppressed)

Before suppression -0.556841745845
After suppression -0.0556841745845


## Benchmark

In [29]:
sample_submission = pd.read_csv('./sample_submission.csv', index_col='fname')
sample_submission['label'] = 'silence'
sample_submission.to_csv('silence_is_golden.csv')

In [30]:
sample_submission.head()

label
fname                      
clip_000044442.wav  silence
clip_0000adecb.wav  silence
clip_0000d4322.wav  silence
clip_0000fb6fe.wav  silence
clip_0001d1559.wav  silence